In [77]:
import pandas as pd

In [78]:
df = pd.read_csv('nba_games.csv', index_col=0)

In [79]:
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9265,240.0,240.0,47.0,89.0,0.528,14.0,31.0,0.452,19.0,25.0,...,41.0,45.3,162.0,135.0,DEN,126,0,2023,2022-12-28,True
9266,240.0,240.0,43.0,87.0,0.494,10.0,27.0,0.370,21.0,27.0,...,33.6,31.0,163.0,123.0,SAS,110,1,2024,2023-11-10,True
9267,240.0,240.0,43.0,92.0,0.467,12.0,34.0,0.353,12.0,19.0,...,51.5,33.0,200.0,112.0,MIN,117,0,2024,2023-11-10,False
9268,240.0,240.0,42.0,82.0,0.512,12.0,27.0,0.444,12.0,19.0,...,53.2,28.0,146.0,126.0,TOR,121,1,2024,2023-11-24,False


In [80]:
df = df.sort_values('date').reset_index(drop=True)

In [81]:
del df['mp.1']
del df['mp_opp.1']
del df['index_opp']

In [82]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

df = df.groupby('team', group_keys=False).apply(add_target)

/var/folders/mf/qs625vr931q7033g8p7tb9k80000gn/T/ipykernel_12450/514775512.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team['target'] = team['won'].shift(-1)
/var/folders/mf/qs625vr931q7033g8p7tb9k80000gn/T/ipykernel_12450/514775512.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team['target'] = team['won'].shift(-1)
/var/folders/mf/qs625vr931q7033g8p7tb9k80000gn/T/ipykernel_12450/514775512.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

In [83]:
df['target'][pd.isnull(df['target'])] = 2

/var/folders/mf/qs625vr931q7033g8p7tb9k80000gn/T/ipykernel_12450/3405584470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][pd.isnull(df['target'])] = 2


In [84]:
df['target'] = df['target'].astype(int, errors='ignore')

In [85]:
nulls = pd.isnull(df)

In [86]:
nulls = nulls.sum()

In [87]:
nulls = nulls[nulls > 0]

In [88]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [89]:
df = df[valid_columns].copy()

In [90]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction='forward', cv=split)

In [91]:
removed_columns = ['season', 'date', 'won', 'target', 'team', 'team_opp']

In [92]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [93]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [94]:
sfs.fit(df[selected_columns], df['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [95]:
predictors = list(selected_columns[sfs.get_support()])

In [96]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data['season'].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        
        train = data[data['season'] < season]
        test = data[data['season'] == season]    
        
        model.fit(train[predictors], train['target'])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        
        combined = pd.concat([test['target'], preds], axis=1)
        combined.columns = ['actual', 'prediction']
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [97]:
predictions = backtest(df, rr, predictors)

In [98]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions['actual'] != 2]
accuracy_score(predictions['actual'], predictions['prediction'])

0.5416274694261524

In [99]:
df.groupby('home').apply(lambda x: x[x['won'] == 1].shape[0] / x.shape[0])

home
0.0    0.440992
1.0    0.559008
dtype: float64

In [101]:
df.dtypes

mp          float64
fg          float64
fga         float64
fg%         float64
3p          float64
             ...   
home_opp    float64
season        int64
date         object
won            bool
target        int64
Length: 140, dtype: object

In [109]:
df_rolling = df[list(selected_columns) + ['won', 'team', 'season']]

def find_team_averages(team):
    rolling = team.rolling(10).mean()
    return rolling

df_rolling = df_rolling.groupby(['team', 'season'], group_keys=False).apply(find_team_averages)

DataError: Cannot aggregate non-numeric type: object

In [110]:
df_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,0.0,0.500000,0.508772,0.478049,0.444444,0.566038,0.449753,0.285714,0.34,0.624286,...,0.039,1.000000,0.178205,0.323232,0.428571,0.349515,1.0,True,LAC,2021
1,0.0,0.357143,0.298246,0.468293,0.259259,0.358491,0.383855,0.523810,0.58,0.677143,...,0.045,0.170860,0.233333,0.262626,0.376623,0.417476,0.0,False,LAL,2021
2,0.0,0.333333,0.614035,0.236585,0.296296,0.433962,0.372323,0.309524,0.42,0.502857,...,0.074,0.408805,0.235897,1.000000,0.168831,0.504854,1.0,False,GSW,2021
3,0.0,0.452381,0.491228,0.439024,0.481481,0.471698,0.579901,0.571429,0.60,0.732857,...,0.127,0.475891,0.129487,0.833333,0.454545,0.252427,0.0,True,BRK,2021
4,0.0,0.357143,0.421053,0.378049,0.333333,0.584906,0.314662,0.523810,0.52,0.795714,...,0.038,0.371069,0.073077,0.409091,0.441558,0.271845,0.0,True,MIN,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9265,0.0,0.452381,0.456140,0.463415,0.333333,0.641509,0.285008,0.309524,0.36,0.642857,...,0.045,0.213836,0.128205,0.297980,0.428571,0.388350,1.0,False,MIL,2024
9266,0.0,0.619048,0.491228,0.624390,0.666667,0.603774,0.657331,0.476190,0.52,0.694286,...,0.038,0.650943,0.112821,0.368687,0.792208,0.621359,1.0,True,GSW,2024
9267,0.0,0.476190,0.508772,0.451220,0.740741,0.735849,0.612850,0.642857,0.60,0.865714,...,0.140,0.213836,0.146154,0.439394,0.740260,0.650485,0.0,False,UTA,2024
9268,0.0,0.190476,0.333333,0.234146,0.222222,0.358491,0.327842,0.452381,0.56,0.571429,...,0.068,0.300839,0.141026,0.398990,0.207792,0.533981,0.0,False,POR,2024


In [111]:
rolling_cols = [f'{col}_10' for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [112]:
df = df.dropna()

In [113]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby('team', group_keys=False).apply(lambda x: shift_col(x, col_name))

df['home_next'] = add_col(df, 'home')
df['team_opp_next'] = add_col(df, 'team_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

In [114]:
full = df.merge(df[rolling_cols + ['team_opp_next', 'date_next', 'team']], 
                left_on=['team', 'date_next'],
                right_on=['team_opp_next', 'date_next'])

In [115]:
removed_columns = list(full.columns[full.dtypes == 'object']) + removed_columns

In [116]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [117]:
sfs.fit(full[selected_columns], full['target'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [118]:
predictors = list(selected_columns[sfs.get_support()])

In [119]:
predictions = backtest(full, rr, predictors)

In [120]:
accuracy_score(predictions['actual'], predictions['prediction'])

0.5747883349012229